# Running Splycer

Here I demonstrate the process of running Splycer from start to finish. First, load in all of the necessary modules.

In [1]:
import sys
import os
import pickle as pkl
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

sys.path.append("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking")
from record_set import RecordDataFrame
from pairs_set import PairsCOO
from feature_engineer import FeatureEngineer
from xgboost_match import XGBoostMatch
os.chdir("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/example")

This next cell creates RecordDataFrame objects. This is essentially a wrapper for a dataframe object. This wrapper standardizes the way that Splycer retrieves records, allowing you to use any data structure that you want if you create a wrapper for it.

In [2]:
#Load in the census data.
delaware_1910 = RecordDataFrame(2, pd.read_csv("delaware_1910.csv", index_col="index"))
delaware_1920 = RecordDataFrame(3, pd.read_csv("delaware_1920.csv", index_col="index"))

In [3]:
delaware_1910.df["first"] = delaware_1910.df["first"].mask(delaware_1910.df["first"].isnull(), "")
delaware_1920.df["first"] = delaware_1920.df["first"].mask(delaware_1920.df["first"].isnull(), "")
delaware_1910.df["last"] = delaware_1910.df["last"].mask(delaware_1910.df["last"].isnull(), "")
delaware_1920.df["last"] = delaware_1920.df["last"].mask(delaware_1920.df["last"].isnull(), "")

In [4]:
delaware_1910.df["first"].isnull().value_counts()

False    203320
Name: first, dtype: int64

Next, we create the pairs set. This represents post-blocking comparisons that we want to make. Once again, there is a wrapper for any data structure that you want. Here, I am using a COO matrix, which is a compressed matrix representation of compares.

In [5]:
#Load in the compares set
compares = pd.read_csv("delaware_compares.csv")
compares = PairsCOO(2,3, compares.index_1910.values, compares.index_1920.values, np.ones(compares.shape[0]))

Constructing the comparison engineer is the most intensive part of the pipeline. It is constructed by incrementally adding comparisons that you want to perform. For each comparison, you must specify the column name(s) of the record you want to compare; the type of comparison; and any extra arguments that the comparison takes.

Example: suppose I wanted to compare first name using jaro-winkler score. After creating a ComparisonEngineer object `comparison_engine`, the code to do this is `comparison_engine.add_comparison("first_name", "jw")`. If I wanted to also use a commonality weight, I would use `comparison_engine.add_comparison("first_name", "jw", {"comm_weight": 'd', "comm_col": "first_comm"})`.

Below, you can see that I generate the comparison pipeline by creating lists of columns, comparisons, and extra arguments. This is the most efficient way of coding up the comparison pipeline.

In [6]:
#Construct the feature engineer
fe = FeatureEngineer()
#FIXME add parent geo-coordinates
cols = ["marstat", "race", "rel", "mbp", "fbp", "first_sdxn", "last_sdxn", "bp", "county",
        "immigration", "birth_year", ["res_lat", "res_lon"], ["bp_lat", "bp_lon"],
        [f"first_vec{i}" for i in range(2, 202)], [f"last_vec{i}" for i in range(2,202)],
        "first", "last",
        "first", "last"
       ]

col_comps = ["exact match"] * 9
col_comps.extend(["abs dist"] * 2)
col_comps.extend(["euclidean dist"] * 4)
#col_comps.extend(["geo dist"] * 2)
col_comps.extend(["jw"] * 2)
col_comps.extend(["trigram"] * 2)

col_args = list({} for i in range(5))
col_args.extend([{"comm_weight": "d", "comm_col": "first_comm"}, {"comm_weight": "d", "comm_col": "last_comm"},
                 {"comm_weight": "d", "comm_col": "bp_comm"}])
col_args.extend(list({} for i in range(7)))
col_args.extend([{"comm_weight": "d", "comm_col": "first_comm"}, {"comm_weight": "d", "comm_col": "last_comm"}] * 2)
assert len(cols) == len(col_comps) == len(col_args)

In [7]:
for i,j,k in zip(cols, col_comps, col_args):
    fe.add_comparison(i, j, k)

In [8]:
#Construct XGBoostMatch
with open("R:/JoePriceResearch/record_linking/projects/deep_learning/ml-record-linking/model.xgboost", "rb") as file:
    model = pkl.load(file)
model.get_booster().feature_names = [f"f{i}" for i in range(19)]

In [9]:
test = XGBoostMatch(delaware_1910, delaware_1920, compares, fe, model)
%prun test.run("test.csv")

R:\JoePriceResearch\Python\AnacondaNew\envs\ml_rec_env\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log1p
  """Entry point for launching an IPython kernel.


Currently can run 3,237 compares/sec. Choosing a different ngram algorithm will lead to the largest optimizing gains.

In [10]:
fe.pipeline

In [26]:
0x00007FFE4099AD30

140729982233904

In [3]:
%load_ext Cython

In [24]:
%%cython --annotate
import abc
import numpy as np
cimport numpy as np

class CompareBase(metaclass=abc.ABCMeta):
    def __init__(self, col):
        self.col = col
    @abc.abstractmethod
    def compare(self, rec1, rec2):
        """Perform the comparison operation"""
    def transform(self, rec1, rec2):
        return self.compare(rec1, rec2)

class WeightedCompareBase(CompareBase, metaclass=abc.ABCMeta):
    def __init__(self, col, comm_weight=None, comm_col=None):
        self.col = col
        self.comm_weight = comm_weight
        self.comm_col = comm_col
        
    def transform(self, rec1, rec2):
        return self.weight(rec1, rec2, self.compare(rec1, rec2) )
    
    def weight(self, rec1, rec2, comp_val):
        if self.comm_weight == "d":
            return comp_val / np.log1p( (rec1[self.comm_col] + rec2[self.comm_col]) / 2 )
        elif self.comm_weight == "m":
            return comp_val * np.log1p( (rec1[self.comm_col] + rec2[self.comm_col]) / 2 )
        return comp_val
cdef np.ndarray[float,ndim=1] l2norm(np.ndarray rec1, np.ndarray rec2):
    cdef int rows = rec1.shape[0]
    cdef int cols = len(rec1[0])
    cdef int i,j
    cdef np.ndarray[np.float32_t,ndim=2] u = np.ndarray((rows,cols), dtype=np.float32)
    for i in range(rows):
        for j in range(cols):
            u[i,j] = rec1[i][j] - rec2[i][j]
    return np.linalg.norm(u, axis=1)

class EuclideanDistance(WeightedCompareBase):
    def __init__(self, col=[f"first_vec{i}" for i in range(200)], comm_weight=None, comm_col=None):
        super().__init__(col, comm_weight=comm_weight, comm_col=comm_col)

    def compare(self, np.ndarray rec1, np.ndarray rec2):
        #return np.linalg.norm(np.array(rec1[self.col].tolist(), np.float32) - np.array(rec2[self.col].tolist(), np.float32), axis=1)
        return l2norm(rec1[self.col], rec2[self.col])
    
rec1 = np.array([('Billy', 'Bob', 'Joe', 1, 2, 45.31, 122.4 , 1.21, 3.2 , 0.87, 112),
       ('Bill', 'Bob', 'Smith', 1, 1, 40.14, 111.39, 1.21, 3.34, 1.01,  87)],
      dtype=[('first', '<U10'), ('middle', '<U10'), ('last', '<U10'), ('state', 'u1'), ('race', 'u1'), ('bp_lat', '<f4'), ('bp_lon', '<f4'), ('first_vec1', '<f4'), ('first_vec2', '<f4'), ('first_vec3', '<f4'), ('first_comm', '<i4')])
rec2 = np.array([('Bill', 'Bob', 'Smith', 1, 1, 40.14, 111.39, 1.21, 3.34, 1.01,  87),
       ('Billy', 'Bob', 'Joe', 1, 2, 45.31, 122.4 , 1.21, 3.2 , 0.87, 112)],
      dtype=[('first', '<U10'), ('middle', '<U10'), ('last', '<U10'), ('state', 'u1'), ('race', 'u1'), ('bp_lat', '<f4'), ('bp_lon', '<f4'), ('first_vec1', '<f4'), ('first_vec2', '<f4'), ('first_vec3', '<f4'), ('first_comm', '<i4')])
ed = EuclideanDistance(["first_vec1", "first_vec2", "first_vec3"])
ed.compare(rec1, rec2)